In [ ]:
pip install datasets

In [ ]:
pip install transformers

In [ ]:
pip install evaluate

In [ ]:
# from google.colab import drive

# drive.mount('/content/drive')

In [ ]:
import pandas as pd
from scipy.special import softmax
!git clone XXXXXXXXXXXXXXXXXXXXXXXXX

fatal: destination path 'Emojimotion' already exists and is not an empty directory.


In [ ]:
test = pd.read_csv('/content/Emojimotion/datasets_tweets/test_v2.csv')
train=pd.read_csv('/content/Emojimotion/datasets_tweets/train_v2.csv')
valid=pd.read_csv('/content/Emojimotion/datasets_tweets/valid_v2.csv')

In [ ]:
train=train[["tweets",'emojis_num']]
valid=valid[["tweets",'emojis_num']]
print(valid)
print(train)
train.head()

                                                  tweets  emojis_num
0                                      gm gm gm floating          15
1      my daughter is months today more months and wi...           4
2      not in the mood for dreamstate but have to go ...           1
3      followed back cant wait to see more of your ad...           1
4      oh ive just seen amigos are off to ascotgod he...          26
...                                                  ...         ...
17275             folklore the long pond studio sessions          36
17276  gm to the around of my most loyal robotic foll...          35
17277                                and down on the log          19
17278  you are all awesome we always love and proud o...           1
17279  sza maxwell tems yoke lore willow amp megan th...           5

[17280 rows x 2 columns]
                                                  tweets  emojis_num
0      thanks for keeping it down to all the current ...          36
1       

,tweets,emojis_num
0,thanks for keeping it down to all the current ...,36
1,this is so salphobic,24
2,good morning my dear,27
3,friend me on facebook,24
4,i am so depressed right now and dont know whys...,25


In [ ]:
from datasets import Dataset
import torch
import torch.nn.functional as F

In [ ]:
train=Dataset.from_dict(train)
test=Dataset.from_dict(test)
valid=Dataset.from_dict(valid)

In [ ]:
from transformers import AutoTokenizer
#tokenizer
#checkpoint1="sshleifer/tiny-gpt2"
checkpoint1="bert-base-uncased"
#checkpoint1="prajjwal1/bert-tiny"
#checkpoint1="prajjwal1/bert-mini"
#checkpoint1='gpt2'
tokenizer = AutoTokenizer.from_pretrained(checkpoint1)
#tokenizer.pad_token = tokenizer.eos_token
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
max_length=512
def tokenize_function(examples):
    return tokenizer(examples['tweets'], padding='max_length', truncation=True, max_length=max_length)

#tokenize train and validation set
train_tokenized_datasets = train.map(tokenize_function, batched=True)
valid_tokenized_datasets = valid.map(tokenize_function, batched=True)

train_tokenized_datasets = train_tokenized_datasets.rename_column('emojis_num', 'label')
valid_tokenized_datasets = valid_tokenized_datasets.rename_column('emojis_num', 'label')

  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

In [ ]:
from transformers import AutoModelForSequenceClassification
#load model and specify the number of expected labels
model = AutoModelForSequenceClassification.from_pretrained(checkpoint1, num_labels=54)
model.config.pad_token_id = model.config.eos_token_id


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
for parameter in model.bert.parameters():
    parameter.requires_grad = False
for i, m in enumerate(model.bert.encoder.layer):
    if i>=6:
      #print(i)
      for parameter in m.parameters():
        parameter.requires_grad = True
# for parameter in model.bert.encoder.parameters():
#     parameter.requires_grad = False
for parameter in model.bert.pooler.parameters():
    parameter.requires_grad = True

In [ ]:
for name, param in model.named_parameters():
     print(name, param.requires_grad)

In [ ]:
# for name, param in model.named_parameters():
#      print(name, param.requires_grad)

In [ ]:
red_hearts=[1, 5, 15, 29, 31] # red_hearts=['💖', '💗', '💕', '💓', '💘']
flowers=[2, 37, 49] # flowers=['🌹', '🌷', '🌺']
yes=[3, 12] # yes=['✅', '👌']
sad=[4, 14, 18, 25, 50, 23]  # sad=['😔', '😞', '😌', '😣', '😫', '😓']
happy=[7, 52, 47, 10]  # happy=['😄', '😃', '😝', '😋']
angry=[24, 38] # angry=['😠', '😡']
kiss=[33, 8, 22, 40]  # kiss=['😻', '😘', '😚', '💋']

In [ ]:
from transformers import TrainingArguments
#save the checkpoints from training
training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate
#accuracy
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics_8(eval_pred):
    logits, labels = eval_pred
    #predictions = np.argmax(logits, axis=-1)
    # print('Hi',len(logits),)
    # print('labels',len(labels))
    # _,pre=torch.topk(F.softmax(logits),8)
    # softmax(logits, axis=0)
    probs = torch.from_numpy(softmax(logits,1))

    values, idx = torch.topk(probs, k=8, axis=-1)
    total_corr=0
    for i in range(len(labels)):
      #print(i)
      if labels[i] in idx[i]:
        total_corr=total_corr+1
    acc=float(total_corr)/len(labels)
    print('accuracy',acc)
  
    return {'accuracy':acc}

In [ ]:
# #top1
# def compute_metrics_1(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     # print('predictions = ',predictions)
#     # print('labels = ',labels)
#     return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer,DataCollatorWithPadding
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
#specify training arguments to monitor the training process
training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch",
                                  save_strategy='epoch',
                                  num_train_epochs=10,
                                  per_device_eval_batch_size=64)

In [ ]:
#training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=valid_tokenized_datasets,
    data_collator=data_collator,
    compute_metrics=compute_metrics_1,
)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("/content/drive/MyDrive/AAA_BERT_54class_top8_partially_freeze")

In [ ]:
#!cp -r "/content/test_trainer/checkpoint-25920" "/content/drive/MyDrive/"